In [4]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import sqlContext.implicits._

val html = scala.io.Source.fromURL("http://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data").mkString
val list = html.split("\n").map{line => line.split(",")}.toSeq
val oldDF = list.toDF()
//val rdds = sc.parallelize(list)

val dF = oldDF.select(
  $"value".getItem(0).as("predict"),
  $"value".getItem(1).as("a"),
  $"value".getItem(2).as("b"),
  $"value".getItem(3).as("c"),
  $"value".getItem(4).as("d"),
  $"value".getItem(5).as("e"),
  $"value".getItem(6).as("f"),
  $"value".getItem(7).as("g"),
  $"value".getItem(8).as("h"),
  $"value".getItem(9).as("i"),
  $"value".getItem(10).as("j"),
  $"value".getItem(11).as("k"),
  $"value".getItem(12).as("l"),
  $"value".getItem(13).as("m"),
  $"value".getItem(14).as("n"),
  $"value".getItem(15).as("o"),
  $"value".getItem(16).as("p"),
  $"value".getItem(17).as("q"),
  $"value".getItem(18).as("r"),
  $"value".getItem(19).as("s"),
  $"value".getItem(20).as("t"),
  $"value".getItem(21).as("u"),
  $"value".getItem(22).as("v")).drop("value").drop("p")
dF.show(5)




+-------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|predict|  a|  b|  c|  d|  e|  f|  g|  h|  i|  j|  k|  l|  m|  n|  o|  q|  r|  s|  t|  u|  v|
+-------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|      p|  x|  s|  n|  t|  p|  f|  c|  n|  k|  e|  e|  s|  s|  w|  w|  w|  o|  p|  k|  s|  u|
|      e|  x|  s|  y|  t|  a|  f|  c|  b|  k|  e|  c|  s|  s|  w|  w|  w|  o|  p|  n|  n|  g|
|      e|  b|  s|  w|  t|  l|  f|  c|  b|  n|  e|  c|  s|  s|  w|  w|  w|  o|  p|  n|  n|  m|
|      p|  x|  y|  w|  t|  p|  f|  c|  n|  n|  e|  e|  s|  s|  w|  w|  w|  o|  p|  k|  s|  u|
|      e|  x|  s|  g|  f|  n|  f|  w|  b|  k|  t|  e|  s|  s|  w|  w|  w|  o|  e|  n|  a|  g|
+-------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
only showing top 5 rows



In [5]:
import org.apache.spark.ml.feature.RFormula
val formula = new RFormula().setFormula("predict ~ a+b+c+d+e+f+g+h+i+j+k+l+m+n+o+q+r+s+t+u+v")
val preparedDF = formula.fit(dF).transform(dF)


In [6]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.feature.VectorIndexer
import org.apache.spark.ml.regression.{RandomForestRegressionModel, RandomForestRegressor}


val featureIndexer = new VectorIndexer().setInputCol("features").setOutputCol("indexedFeatures").setMaxCategories(12).fit(preparedDF)
val Array(trainingData, testData) = preparedDF.randomSplit(Array(0.7, 0.3))
val rf = new RandomForestRegressor().setLabelCol("label").setFeaturesCol("indexedFeatures")
val pipeline = new Pipeline().setStages(Array(featureIndexer, rf))

// Train model. This also runs the indexer.
val model = pipeline.fit(trainingData)

// Make predictions.
val predictions = model.transform(testData)

// Select example rows to display.
predictions.select("prediction", "label", "features").show(5)
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println("Root Mean Squared Error (RMSE) on test data = " + rmse)



+--------------------+-----+--------------------+
|          prediction|label|            features|
+--------------------+-----+--------------------+
|0.005355146200169348|  0.0|(95,[3,6,12,22,26...|
|0.005355146200169348|  0.0|(95,[3,6,12,22,26...|
|0.005355146200169348|  0.0|(95,[3,6,12,22,26...|
|0.005355146200169348|  0.0|(95,[3,6,12,22,26...|
|0.005355146200169348|  0.0|(95,[3,6,12,22,26...|
+--------------------+-----+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 0.06261856603668746


In [7]:
val predictedLabel = predictions.
        select("label", "prediction").
        withColumn("prediction", 
        when(col("prediction") > 0.5, 1).otherwise(0))
val TP = predictedLabel.filter("label=1").filter("prediction=1").count()
val TN = predictedLabel.filter("label=0").filter("prediction=0").count()
val FP = predictedLabel.filter("label=0").filter("prediction=1").count()
val FN = predictedLabel.filter("label=1").filter("prediction=0").count()
val total = predictedLabel.count()
println("Total test samples: "+ total)
println("True positive: "+ TP)
println("TRue negative: "+ TN)
println("False positive: "+ FP)
println("False negative: "+ FN)
println("Misclassification rate: " + math.round((FP+FN)*100000.0/total.toDouble)/1000.0+"%")

Total test samples: 2382
True positive: 1179
TRue negative: 1201
False positive: 0
False negative: 2
Misclassification rate: 0.084%


In [8]:
model.save("mushroomModel")

In [9]:
import java.io._
val pw = new PrintWriter(new File("mushroomConfusionMetrics.txt" ))
val metrics = Array(TP, TN, FP, FN)
pw.write("TP, TN, FP, FN \n")
pw.write(metrics.mkString(" "))
pw.close